In [21]:
import re
import time
import datetime

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from models import *

import os
import sys
import warnings
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesse
 
from dataset import engine, Churn, Engagement, RFM
# from configs import Config
from sklearn.model_selection import GridSearchCV
# from models import pipeline, parameters

In [22]:
eg = Engagement(engine, task='classification', col_type = 'market', y_col = 'conversion')

In [23]:
eg.df

,customerid,age,gender,income,campaignchannel,campaigntype,adspend,clickthroughrate,conversionrate,websitevisits,pagespervisit,timeonsite,socialshares,emailopens,emailclicks,previouspurchases,loyaltypoints,conversion
0,8000,56,F,136912.0,Social Media,Awareness,6497.870068,0.043919,0.088031,0,2.399017,7.396803,19,6,9,4,688,1
1,8001,69,M,41760.0,Email,Retention,3898.668606,0.155725,0.182725,42,2.917138,5.352549,5,2,7,2,3459,1
2,8002,46,F,88456.0,PPC,Awareness,1546.429596,0.277490,0.076423,2,8.223619,13.794901,0,11,2,8,2337,1
3,8003,32,F,44085.0,PPC,Conversion,539.525936,0.137611,0.088004,47,4.540939,14.688363,89,2,2,0,2463,1
4,8004,60,F,83964.0,PPC,Conversion,1678.043573,0.252851,0.109940,0,2.046847,13.993370,6,6,6,8,4345,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,15995,21,M,24849.0,Email,Awareness,8518.308575,0.243792,0.116773,23,9.693602,14.227794,70,13,6,7,286,0
7996,15996,43,F,44718.0,SEO,Retention,1424.613446,0.236740,0.190061,49,9.499010,3.501106,52,13,1,5,1502,0
7997,15997,28,F,125471.0,Referral,Consideration,4609.534635,0.056526,0.133826,35,2.853241,14.618323,38,16,0,3,738,1
7998,15998,19,F,107862.0,PPC,Consideration,9476.106354,0.023961,0.138386,49,1.002964,3.876623,86,1,5,7,2709,1


In [24]:
eg.df.dtypes

customerid             int64
age                    int64
gender                object
income               float64
campaignchannel       object
campaigntype          object
adspend              float64
clickthroughrate     float64
conversionrate       float64
websitevisits          int64
pagespervisit        float64
timeonsite           float64
socialshares           int64
emailopens             int64
emailclicks            int64
previouspurchases      int64
loyaltypoints          int64
conversion             int64
dtype: object

In [36]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
# cat_cols = eg.df.select_dtypes(include=['object', 'category']).columns
# le = LabelEncoder()
# for col in cat_cols:
#     eg.df[col] = le.fit_transform(eg.df[col])
# eg.df

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_cols = eg.df.select_dtypes(include=['float64']).columns
for col in num_cols:
    eg.df[col] = scaler.fit_transform(eg.df[[col]])

eg.df
num_cols = ['age']
cat_cols = ['gender']
X_train = eg.df[['age', 'gender']]
y_train = eg.df['conversion']

In [37]:
from sklearn.compose import ColumnTransformer

# numerical_columns =['age']
preprocess = ColumnTransformer(transformers=[
			("num_pipeline", MinMaxScaler(), num_cols),
			("cat_pipeline", OneHotEncoder(), cat_cols),
	])

pipeline = Pipeline([
    # scaling
    ('preprocess', preprocess),
    # recode
    #('clf', CLFSwitcher()),
])
pipeline.fit(X_train, y_train)
print(pipeline.fit_transform(X_train, y_train))

[[0.74509804 1.         0.        ]
 [1.         0.         1.        ]
 [0.54901961 1.         0.        ]
 ...
 [0.19607843 1.         0.        ]
 [0.01960784 1.         0.        ]
 [0.25490196 1.         0.        ]]
